# Разработка сентимент-анализа под задачу заказчика

In [35]:
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

import re
import requests
from bs4 import BeautifulSoup

Сперва необходимо посмотреть отзывы, которые предоставлены в качестве тестовой выборки: 

In [26]:
f = open('test.csv', 'r')
print(f.readline())

<review>Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.



Видим, что в датасете идет речь про телефоны, поэтому попробуем напарсить похожих отзывов с существующих открытых ресурсов, а именно с сайта: https://torg.mail.ru/review/goods/mobilephones/

In [29]:
req = requests.get('https://torg.mail.ru/review/goods/mobilephones/')
soap = BeautifulSoup(req.text, "html.parser")

Так как на странице используется пагинация, сперва получим количество страниц:

In [30]:
pages = soap.find('div', {'class': 'pager-info'})
pages.text

'1 - 20 из 18 559'

Видим что у нас 18 548 страниц, теперь попробуем спарсить отзывы с первой страницы: 

In [31]:
section = soap.find('section', {'class': 'card__responses js-review_list js-ustat_container js-ustat_container_reviewsList'})
section_soap = BeautifulSoup(str(section), "html.parser")
review_item__info = section_soap.find_all('div', {'class': 'review-item__body'})

Пройдемся по ним и проверим, правильно ли мы собрали отзывы: 

In [32]:
for i in review_item__info:
    i_soap = BeautifulSoup(str(i), "html.parser")
    score = i_soap.find('span', {'class': 'review-item__rating-counter'}).text
    if i_soap.find('a', {'class': 'more'}) is not None: 
        text = i_soap.find('a', {'class': 'more'}).get('full-text')
    else:
        text = i_soap.find('span', {'class': 'js-more-text'}).text 
    print(score, ' - ', text)

4  -  В целом не плохой телефон. Сочный и яркий экран, на улице всё разборчиво видно в солнечный день. Никаких подвисаний при открытии нескольких приложений сразу. Камера также хорошо фотографировует, но в тёмное время суток так себе конечно фото. Ну и царапается быстро, то есть и товарный вид быстрее потеряет. В остальном все не плохо
5  -  Считаю его одним из лучших в своем ценовом диапазоне. Очень шустрый телефон, приятная оболочка ( лучше гуголовской), хорошо сидит в руке. Отдельно хотел похвалить аккумулятор - телефон спокойно хватает на два дня при среднем времени использования. Единственным недостатком считаю камеру, так как она плохо фокусируется и часто все мажет.
3  -  первый сони использовал пол года, пока не разбил ёмкостный экран, сдавал в сервис, специалистов нет, у товарища такая же проблема. Купил ХА1, телефон хороший, батарей неплохая.												
1  -  Такой телефон и телефоном назвать-то нельзя.Прежде чем захотеть кому-то позвонить,7 раз подумаешь хватит ли у тебя не

Подготовим метод, который запишет распаршенные отзывы в файл: 

In [33]:
def write_to_file(filename, text, score):
    f = open(filename, 'a')
    if int(score[0]) == 3:
        return None
    score = 0 if int(score[0]) <= 3 else 1
    text = re.sub('<[^<]+?>', '', text.replace('\n', ' ').replace('\r', ' '))
    text = re.sub('^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    f.write(text.rstrip() + '	' + str(score) + '\n')
    f.close() 

По количеству страниц понятно, что лучше всего здесь использовать мультипроцессинг, поэтому сделаем метод, который принимает два числа: страница, с которой нужно начать парсинг и страница, которой нужно его закончить: 

In [34]:
def start_parsing(page_start, page_end, filename):
    while page_start <= page_end:
        page_start += 1

        req = requests.get('https://torg.mail.ru/review/goods/mobilephones/' + '?page=' + str(page_start))
        if req.status_code != 200:
            time.sleep(30)
            continue

        soap = BeautifulSoup(req.text, "html.parser")
        section = soap.find('section',
                        {'class': 'card__responses js-review_list js-ustat_container js-ustat_container_reviewsList'})
        section_soap = BeautifulSoup(str(section), "html.parser")
        review_item__info = section_soap.find_all('div', {'class': 'review-item__body'})

        for i in review_item__info:
            i_soap = BeautifulSoup(str(i), "html.parser")
            score = i_soap.find('span', {'class': 'review-item__rating-counter'}).text

            if i_soap.find('a', {'class': 'more'}) is not None:
                text = i_soap.find('a', {'class': 'more'}).get('full-text')
            else:
                text = i_soap.find('span', {'class': 'js-more-text'}).text
            write_to_file(filename, text, score)

Вызовем в директории файл с этим методом и подождем, пока спарсятся отзывы.

После того как данные спарсились, мы можем приступить к выбору классификатора. Загрузим данные в датафрейм, предварительно их обработав текст: 

In [36]:
f = open('reviews.txt', 'r')
f_1 = open('reviews_1.txt', 'w')

for i in f.readlines():
    if len(i.split()) < 100:
        f_1.write(i.replace('	' + i[len(i)-2], '').replace('\n', '').replace('\t', '') + '	' + i[len(i)-2] + '\n')
df = pd.read_csv('reviews_1.txt', sep='	', names=['text', 'target'])
df = df[pd.notnull(df['text'])]

Теперь посмотрим на данные, а именно посчитаем количество положительных и отрицтельных отзывов:

In [3]:
print('pos: ', len(df[(df.target == 1)]))
print('neg: ', len(df[(df.target == 0)]))

pos:  11466
neg:  1535


Видимо, что положительных отзывов намного больше, чем отрицательных, поэтому велик риск, что классификатор будет работать некорректно, подкорректируем датафрейм, удалим часть положительных отзывов:

In [4]:
df = pd.concat([df[(df.target == 1)][:2550], df[(df.target == 0)]])

Теперь выберем классификатор: 

In [5]:
def score(clf):
    scores = cross_val_score(clf, df['text'], df['target'])
    print("Score = {:.5f}".format(scores.mean()))

In [6]:
for classf in [LogisticRegression, SGDClassifier, LinearSVC, MultinomialNB, RandomForestClassifier]:
    clf = make_pipeline(CountVectorizer(ngram_range=(1,2)), classf())
    score(clf)

Score = 0.74665


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.5/dist-packages/s

Score = 0.74175
Score = 0.73098
Score = 0.77040
Score = 0.69499


Видим, что лучшее качество у классификатора MultinomialNB, его и будем использовать в дальнейшем.

Обучим классификатор на наших данных: 

In [7]:
clf = make_pipeline(CountVectorizer(ngram_range=(1,2)), MultinomialNB())
clf.fit(df['text'], df['target'])

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

Загрузим тестовые данные: 

In [8]:
# откроем файл и запигшем весь текст в переменную
f_1 = open('test.csv', 'r')
str_file = ''
for i in f_1.readlines():
    str_file += i

# пройдемся по тексту и отформатируем текст
test_arr = []
for i in str_file.split('<review>'):
    i = i.replace('</review>', '')
    i = re.sub('<[^<]+?>', '', i.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('   ', ''))
    test_arr.append(i)

test_arr.remove('')
# создадим датафрейм
df_test = pd.DataFrame(data={'text': test_arr})
df_test = df_test[pd.notnull(df_test['text'])]

Запишем результаты модели на тестовых данных: 

In [23]:
def write_answer(pred_arr):
    f = open('sub.csv', 'w')
    count = 0
    f.write('Id,y' + '\n')
    for i in pred_arr:
        f.write(str(count) + ',' + 'pos' + '\n') if i == 1 else f.write(str(count) + ',' + 'neg' + '\n')
        count += 1
        
pred_arr = clf.predict(df_test['text'])
write_answer(pred_arr)

Результат составил 74%. Как-минимум необходимо набрать 85%, поэтому нам необходимо, либо подобрать другой классификатор, либо лучше обучить текущий. Попробуем поработать со второй гипотезой и обучить классификатор на большем массиве данных. Для решения этой задачи мы можем, либо напарсить еще отзывов, либо попробовать поискать отзывы в открытых источниках, например на Kaggle. После небольшого поиска удалось найти датасет: https://www.kaggle.com/alxmamaev/market-comments-tonality-analys, а также https://www.kaggle.com/thorinhood/russian-twitter-sentiment. В первом датасете мы видим отзывы на товары, во втором твиты. 

Попробуем дообучить нашу модель на новых данных, для этого загрузим данные и объеденим их в единый датафрейм: 

Начнем с твитов: 

In [10]:
df_tw = pd.read_csv('tw_neg.csv', sep=';', names=['item_id1', 'item_id2', 'author', 'text',"id_1","id_2","id_3","id_4","id_5","id_6","id_7","id_8"])
df_tw['target'] = 0
df_tw = df_tw.drop(['item_id1', 'item_id2','author', "id_1","id_2","id_3","id_4","id_5","id_6","id_7","id_8"], axis=1)
df_tw.head()

,text,target
0,на работе был полный пиддес :| и так каждое за...,0
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",0
2,@elina_4post как говорят обещаного три года жд...,0
3,"Желаю хорошего полёта и удачной посадки,я буду...",0
4,"Обновил за каким-то лешим surf, теперь не рабо...",0


In [11]:
df_tw_2 = pd.read_csv('tw_pos.csv', sep=';', names=['item_id1', 'item_id2', 'author', 'text',"id_1","id_2","id_3","id_4","id_5","id_6","id_7","id_8"])
df_tw_2['target'] = 1
df_tw_2 = df_tw_2.drop(['item_id1', 'item_id2','author', "id_1","id_2","id_3","id_4","id_5","id_6","id_7","id_8"], axis=1)
df_tw_2.head()

,text,target
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [12]:
df_twt = pd.concat([df_tw_2, df_tw])

Теперь добавим данные отзывов на товары: 

In [13]:
df_market = pd.read_csv('market.csv', sep=',', names=['item_id1', 'item_id2', 'brand', 'user_id', 'date', 'comment', 'reting', 'negative_comment', 'positive_comment'])

In [14]:
m_neg = pd.concat([df_market[df_market.reting=='1.0'], df_market[df_market.reting=='2.0']])
m_neg['target'] = 0
m_neg = m_neg.drop(['item_id1', 'item_id2','brand', 'user_id', 'date', 'reting', 'negative_comment', 'positive_comment'], axis=1)
m_neg.rename(index=str, columns={"comment": "text"})
m_pos = pd.concat([df_market[df_market.reting=='4.0'], df_market[df_market.reting=='5.0']])[:2000]
m_pos['target'] = 1
m_pos = m_pos.drop(['item_id1', 'item_id2','brand', 'user_id', 'date', 'reting', 'negative_comment', 'positive_comment'], axis=1)
m_pos.rename(index=str, columns={"comment": "text"})
m_df = pd.concat([m_neg, m_pos]).rename(index=str, columns={"comment": "text"})
m_df.head()

,text,target
6,Сегодня купила 2 таких вентилятора! Ужасный. С...,0
11,"Смотрели сегодня эту модель в магазине, сначал...",0
18,"Ужасная некачественная сборка, мне попался с н...",0
29,"Отвратительный пылесос, лучше бы мы не покупал...",0
52,"Был у меня такой тепловентилятор, проработав н...",0


Объеденим теперь эти два датафрейма в один: 

In [15]:
df = pd.concat([df_twt, m_df, df])

Теперь попробуем еще раз обучить классификатор и проверить его на тестовой выборке:

In [20]:
clf = make_pipeline(CountVectorizer(ngram_range=(1,2)), MultinomialNB())
clf.fit(df['text'], df['target'])

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

Запишем опять данные в файл и отправим его на Kaggle: 

In [24]:
pred_arr = clf.predict(df_test['text'])
write_answer(pred_arr)

### На этот раз нам удалось получить качество 86%, что превышает минимальный показатель 85%.

Можно поработать над улучшение качества нашей модели. Попробуем добавить в наш пайплан transformer:

In [39]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline


def text_classifier(vectorizer, transformer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )

In [40]:
clf = text_classifier(TfidfVectorizer(ngram_range=(1, 2)),
                               TfidfTransformer(),
                               MultinomialNB())

In [42]:
clf = clf.fit(df['text'], df['target'])

In [44]:
pred_arr = clf.predict(df_test['text'])
write_answer(pred_arr)

### С даннным классификатором у нас получилось качество 92%.


In [47]:
import pickle
pickle.dump( clf, open( "clf.p", "wb" ) )